<a href="https://colab.research.google.com/github/JacobSchmitt/Basaa-Machine-Learning/blob/main/basaa/test/Basaa_bnt_to_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/train.tsv 
!wget https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/dev.tsv 
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/convert.py
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/Makefile
!wget https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/evaluate.py

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving corr_dev.tsv to corr_dev.tsv
Saving corr_train.tsv to corr_train.tsv


In [ ]:
%%capture
!pip install simpletransformers

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
train_df = pd.read_csv('corr_train.tsv', sep ='\t', names = ['input_text', 'target_text']).astype(str)
eval_df = pd.read_csv('corr_dev.tsv', sep ='\t', names = ['input_text', 'target_text']).astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

In [ ]:
def low(x):
  return x.lower()

In [ ]:
train_df.input_text = train_df.input_text.apply(low)
train_df.target_text = train_df.target_text.apply(low)

eval_df.input_text = eval_df.input_text.apply(low)
eval_df.target_text = eval_df.target_text.apply(low)

In [ ]:
import logging
from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Apr  8 14:03:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
import shutil

dir_path = '/content/outputs'

try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))


In [ ]:
model_args = {
    "preprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 60,
    "train_batch_size": 10,
    "num_train_epochs": 20,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    # "silent": True,
    "evaluate_generated_text": True,
    "evaluate_during_training": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": True,
    "save_best_model": True,
    "use_multiprocessed_decoding": True,
    'no_cache': True,
    # 'max_length': 512,
    'fp16': False
}

model = Seq2SeqModel(encoder_decoder_type = "marian", encoder_decoder_name= 'Helsinki-NLP/opus-mt-bnt-en', args=model_args)

In [ ]:
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/10000 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.7869268792867661}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of Helsinki-NLP/opus-mt-bnt-en model complete. Saved to outputs/.


(20000,
 {'eval_loss': [1.112243103981018,
   0.8165746994018555,
   0.8165746994018555,
   0.7454523603916168,
   0.7520925040245057,
   0.7520925040245057,
   0.6700789542198181,
   0.6891730804443359,
   0.6891730804443359,
   0.7058753213882446,
   0.7389250996112824,
   0.7389250996112824,
   0.7017594475746155,
   0.7084268958568573,
   0.7084268958568573,
   0.7349517855644226,
   0.7376291402578354,
   0.7376291402578354,
   0.7610862965583801,
   0.7542951231002808,
   0.7542951231002808,
   0.7655778396129608,
   0.7685506361722946,
   0.7685506361722946,
   0.7825214645862579,
   0.7849053245782852,
   0.7849053245782852,
   0.7879737694263458,
   0.7869268792867661,
   0.7869268792867661],
  'global_step': [1000,
   2000,
   2000,
   3000,
   4000,
   4000,
   5000,
   6000,
   6000,
   7000,
   8000,
   8000,
   9000,
   10000,
   10000,
   11000,
   12000,
   12000,
   13000,
   14000,
   14000,
   15000,
   16000,
   16000,
   17000,
   18000,
   18000,
   19000,
   2000

In [ ]:
import logging
import pandas as pd

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = {
'max_length': 512,
'length_penalty' : 1,
'num_beams' : 10}

model = Seq2SeqModel(encoder_decoder_type = "marian", encoder_decoder_name= 'outputs/checkpoint-20000', args=model_args) 
# /content/outputs/checkpoint-20000

In [ ]:
sinhala_truth = eval_df['target_text'].tolist()

In [ ]:
sinhala_preds = model.predict(eval_df['input_text'].tolist())

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
df1 =  pd.DataFrame({'input': eval_df['input_text'], 'pred':sinhala_preds})

In [ ]:
def smart(x):
  double_e = 'e'+'́'
  double_o = 'o' + '̂'
  double_o_dots = 'o' + '̈'
  y = x.replace(double_e, 'é').replace(double_o, 'ô').replace(double_o_dots, 'ö')
  y = y.replace('▁', ' ')
  y = y.replace('_', ' ')
  y = y.capitalize()
  return y

In [ ]:
df1.pred = df1.pred.apply(smart)

In [ ]:
eval_df.input_text = eval_df.input_text.apply(smart)
eval_df.target_text = eval_df.target_text.apply(smart)

In [ ]:
eval_df.to_csv('test.tsv', sep='\t', index=False, header=False)
df1.to_csv('seq_bnt.tsv', sep='\t', index=False, header=False)

In [ ]:
df1.head()

,input,pred
0,Malét a mbôŋe hyémbi.,Màlêt à môŋɛ hyembi.
1,A ntuhul me két (i) nuga.,Mɛ̀ tuhul mɛ̀ kêt (i) nugā.
2,"Ñkum u ŋkôs ndoŋ, ŋkum u nlôôs ndoŋ.","Ncɔ̀m u ŋkos ndɔŋ, ŋ̀m u nlòos ndɔŋ."
3,Maŋ mut unu u nlo u mboda nyono wotama.,Mùŋ mùt unu u nlɔ̀ u mɓɔ̀da nyɔ̀nɔ wɔtāma.
4,Boŋge ba ŋke jogop ba kahal ndugi kôt maŋgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ̀ jɔgɔp ɓa kahal ndugi kot màŋgùdgùdgà.


In [ ]:
!python3 evaluate.py test.tsv seq_bnt.tsv

CER: 25.735948574254163
WER: 67.88239735511937
